### Reports Automation
- We take sentiment scores and predicted categories against each sentences and prepare this data to align with the template that we use in Gramex for automation
- The below code cells should be executed in sequence (Step 1 to Step 10)

**Prepare Data**

In [2]:
# load libraries
import pandas as pd 

In [ ]:
# load data
df = pd.read_csv('../output/step2_output_sentiment_scores_categories.csv')

##### Step 1

In [ ]:
cols_to_read = ['d_pol', 'doc_id', 'company_name', 'sents', 's_pol', 'category']
dt = pd.read_csv('step2_output_sentiment_scores_categories.csv', usecols = cols_to_read)
dt['d_pol'] = dt.groupby('doc_id')['d_pol'].fillna(method = 'ffill')

##### Step 2

In [ ]:
dt['group_index'] = dt.groupby('company_name').cumcount() + 1 # make index sequential by group
dt = dt.sort_values(['company_name', 'group_index'], ascending = True)

##### Step 3

In [ ]:
# group every 4th row
# This is to limit 4 sentences to appear in each slides
counter = 1
for rid, row in dt.iterrows():
    if dt.loc[rid, 'group_index'] % 4 == 0:
        dt.loc[rid, 'group'] = counter
        counter += 1
    else:
        next 

##### Step 4

In [ ]:
# fillna within a group
dt['group'] = dt.groupby('company_name')['group'].fillna(method = 'bfill')

##### Step 5

In [ ]:
# fill value for left out element in each group
max_group_rank = pd.DataFrame(dt.groupby('company_name')['group'].max())
max_group_rank = max_group_rank[max_group_rank['group'].notnull()]
max_group_rank = max_group_rank['group'].astype(int)
dt = pd.merge(dt, max_group_rank, how = 'left', left_on = 'company_name', right_index = True)

##### Step 6

In [ ]:
dt['group_y'] = dt['group_y'].apply(lambda x: str(x) + '.1')
dt['group_x'] = dt['group_x'].astype('str')
dt['group_x'] = np.where(dt['group_x'] == 'nan', dt['group_y'], dt['group_x'])
dt = dt[dt['group_x'] != 'nan.1']

In [ ]:
# Group predicted categories into a pre-defined categories 
g1 = ['New Projects/Market Expansion/Capex Plans', 'Business Risk/General']
dt['categories'] = dt['category'].apply(lambda x: g1[0] if x in ['New Projects', 'Market Expansion', 'Capex Plans'] else (g1[1] if x in ['Business Risk', 'General'] else x))

In [ ]:
dt['sno'] = dt.groupby(['group_x', 'categories']).cumcount() + 1

##### Step 7

In [ ]:
# Balance Categories
groups_to_have = ['New Projects/Market Expansion/Capex Plans', 'Business Risk/General', 'Business Performance']
groups_to_have.sort()
grouped = dt.groupby('group_x')
group_df = pd.DataFrame()
for gid, group in grouped:
    max_cat = group.categories.value_counts().count()
    if max_cat < 3:
        left_out = list(set(groups_to_have) - set(group['categories'].unique().tolist()))
        for cat in left_out:
            g = pd.DataFrame(group).reset_index()
            g.index = g.index.astype(str)
            g.loc[str(g.index.to_list()[-1]) + '_'  + str(left_out.index(cat))] = [0, g.iloc[-1, 1], '', g.iloc[-1, 3], '', '', '', '', gid, '', str(cat), '']
            group_df = group_df.append(g)

##### Step 8

In [ ]:
# Balance number of rows in each category
# to find out maximum number of rows that one category is having in our dataset
max_rows = []
grouped = group_df.groupby('group_x')
for gid, group in grouped:
    max_row = pd.DataFrame(group)['categories'].value_counts().values
    max_row = np.amax(max_row).item()
    max_rows.append(max_row)
max_row_value = max(max_rows)
print(max_row_value)

complete_data_frame = pd.DataFrame()
for gid, group in grouped:
    g = pd.DataFrame(group)
    cat_count_list = pd.DataFrame(group)['categories'].value_counts().sort_index(ascending = True).to_dict()
    for key, value in cat_count_list.items():
        if value != 8:
            for i in range(value, max_row_value):
                g.loc[str(g.index.to_list()[-1]) + '_'  + str(i)] = [0, g.iloc[-1, 1], '', g.iloc[-1, 3], '', '', '', '', gid, '', str(key), '']
        else:
            next
    complete_data_frame = complete_data_frame.append(g)

##### Step 9

In [ ]:
# remove columns 
complete_data_frame.drop(['doc_id', 'group_index', 'group_y', 'index', 'sno'], axis = 1, inplace = True)
complete_data_frame.sort_values(by = 'group_x', inplace = True)

##### Step 10

In [ ]:
# create row number in each group items
# write final dataframe into an excel 
complete_data_frame['sno'] = complete_data_frame.groupby(['group_x', 'categories']).cumcount() + 1
complete_data_frame.to_excel('automation/data_for_automation_updated.xlsx', index = False)